<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/TestingStatsNFeaturesCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Refactor GatheringBaseMetrics

Used to test code built up/ worked out in the following notebook...
* https://github.com/curtiscu/LYIT/blob/master/GatheringBaseMetrics_1.ipynb

# Setup env


In [1]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 36
-rw------- 1 root root 21765 Jun 26 11:24 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__
-rw------- 1 root root  1931 Jun 26 11:27 stats_n_features.py
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [4]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

     |████████████████████████████████| 61kB 1.9MB/s 


In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

>> LOADING custom module, when: 2020-06-26 11:28:58.021182, module name: data_prep


In [7]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function in data_prep.py called and worked! when: 2020-06-26 11:28:58.070556,  param:bling


In [14]:
import stats_n_features as sf

In [15]:
sf.test_function_call('hello')

Test function in stats_n_features called and worked! when: 2020-06-26 11:32:28.561504,  param:hello


## Pandas display options

In [9]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Testing stats_n_features.py

## Load some data

In [16]:
file_1 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'
file_7 = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer7/eval_session/1_funk-groove1_138_beat_4-4.mid'

file_df, f, mtt = dp.load_file(file_1)  # load data

>> LOADING custom module, when: 2020-06-26 12:46:06.860626, module name: stats_n_features
FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 1300 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'MIDI' 1300 messages>]
    MIDI file type: 0
    > processing track: <midi track 'MIDI' 1300 messages>
    note_on span - first tick: 3 , last tick: 30658 
    good instruments: 6, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 42.0: 'Closed Hi Hat (42)', 43.0: 'High Floor Tom (43)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    bar info - bars in file: 16, bar quantize

## (1) Test sf.frequency_table()

In [36]:
freq_table = sf.frequency_table(file_df)

>> LOADING custom module, when: 2020-06-26 12:52:20.065192, module name: stats_n_features


In [37]:
display(freq_table)

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
note,,,,,,,,,,,,,,,,
Bass Drum 1 (36),12,0,16,4,0,0,0,0,0,0,15,6,0,1,0,0
Acoustic Snare (38),10,5,0,0,16,0,0,16,0,17,0,2,8,0,8,19
Closed Hi Hat (42),15,2,11,0,16,2,14,0,17,3,13,0,16,1,13,1
High Floor Tom (43),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Crash Cymbal 1 (49),1,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
Ride Cymbal 1 (51),16,0,16,3,16,0,16,0,17,0,12,0,16,0,16,0


## (2) Test sf.error_buckets()

In [38]:
err_bucks = sf.error_buckets(file_df)

In [40]:
display(err_bucks)

track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
1          9               42            0:41          908        77   
                           51            0:43          940        46   
                           42            0:45          971         0   
                           51            0:46          971         0   
2          16              38           0:155         3710        37   
                           38           0:158         3747        41   
3          1               38           0:161         3796        33   
                           38           0:168         3846        23   
4          10              38           0:289         6813        37   
                           38           0:290         6867        20   
5          16              38           0:401         9468        37   
                           38           0:404         9506        33   
                           38           0:406         9532        30   
6          16              38           0:487        11397        36   
                           38           0:491        11441        32   
9          16              38           0:730        17147        38   
                           38           0:733        17182        38   
10         16              38           0:814        19056        43   
                           38           0:817        19092        46   
                           38           0:820        19116        40   
13         16              38          0:1056        24797        33   
                           38          0:1059        24845        32   
                           38          0:1062        24876        32   
14         16              38          0:1140        26733        40   
                           38          0:1143        26778        41   
                           38          0:1145        26805        34   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          9               42            -52         960           1   
                           51            -20         960           1   
                           42             11         960           1   
                           51             11         960           1   
2          16              38            -10        3720           2   
                           38             27        3720           2   
3          1               38            -44        3840           3   
                           38              6        3840           3   
4          10              38            -27        6840           4   
                           38             27        6840           4   
5          16              38            -12        9480           5   
                           38             26        9480           5   
                           38             52        9480           5   
6          16              38             -3       11400           6   
                           38             41       11400           6   
9          16              38            -13       17160           9   
                           38             22       17160           9   
10         16              38            -24       19080          10   
                           38             12       19080          10   
                           38             36       19080          10   
13         16              38            -43       24840          13   
                           38              5       24840          13   
                           38             36       24840          13   
14         16              38            -27       26760          14   
                           38             18       26760          14   
                           38             45       26760          14   

In [45]:
err_bucks.sample(3)

,,,track_msg_num,total_ticks,velocity,beat_offset,beat_center,bar_number,bar_beat_number
bar_number,bar_beat_number,note,,,,,,,
14,16,38,0:1140,26733,40,-27,26760,14,16
10,16,38,0:814,19056,43,-24,19080,10,16
6,16,38,0:487,11397,36,-3,11400,6,16


## (3) Test sf.gather_stats()

In [51]:
stats_dict = sf.gather_stats(file_df)

>> LOADING custom module, when: 2020-06-26 13:48:18.503553, module name: stats_n_features
>> Gathering stats for instrument: Bass Drum 1 (36)
>> Gathering stats for instrument: Acoustic Snare (38)
>> Gathering stats for instrument: Closed Hi Hat (42)
>> Gathering stats for instrument: High Floor Tom (43)
>> Gathering stats for instrument: Crash Cymbal 1 (49)
>> Gathering stats for instrument: Ride Cymbal 1 (51)


In [55]:
for key in stats_dict.keys():
  print('Showing stats for: {}'.format(key))
  display(stats_dict[key])

Showing stats for: 36


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,12.00,0.0,16.00,4.00,0.0,0.0,0.0,0.0,0.0,0.0,15.00,6.00,0.0,1.0,0.0,0.0
off_min,-25.00,0.0,-29.00,-20.00,0.0,0.0,0.0,0.0,0.0,0.0,-54.00,-42.00,0.0,6.0,0.0,0.0
off_max,10.00,0.0,6.00,5.00,0.0,0.0,0.0,0.0,0.0,0.0,6.00,-6.00,0.0,6.0,0.0,0.0
off_median,-8.00,0.0,-10.00,-8.50,0.0,0.0,0.0,0.0,0.0,0.0,-26.00,-13.50,0.0,6.0,0.0,0.0
off_mean,-9.25,0.0,-10.56,-8.00,0.0,0.0,0.0,0.0,0.0,0.0,-25.87,-17.33,0.0,6.0,0.0,0.0
off_iqr,18.00,0.0,17.00,21.00,0.0,0.0,0.0,0.0,0.0,0.0,28.00,10.00,0.0,0.0,0.0,0.0
off_std,12.14,0.0,10.75,12.36,0.0,0.0,0.0,0.0,0.0,0.0,18.63,13.35,0.0,0.0,0.0,0.0
vel_min,50.00,0.0,42.00,48.00,0.0,0.0,0.0,0.0,0.0,0.0,46.00,48.00,0.0,52.0,0.0,0.0
vel_max,69.00,0.0,125.00,57.00,0.0,0.0,0.0,0.0,0.0,0.0,127.00,65.00,0.0,52.0,0.0,0.0
vel_median,61.50,0.0,60.50,54.50,0.0,0.0,0.0,0.0,0.0,0.0,59.00,56.50,0.0,52.0,0.0,0.0


Showing stats for: 38


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,10.00,5.00,0.0,0.0,16.00,0.0,0.0,16.00,0.0,17.00,0.0,2.00,8.00,0.0,8.00,19.00
off_min,-58.00,-38.00,0.0,0.0,-24.00,0.0,0.0,-44.00,0.0,-28.00,0.0,7.00,-24.00,0.0,-40.00,-43.00
off_max,55.00,-3.00,0.0,0.0,16.00,0.0,0.0,10.00,0.0,27.00,0.0,55.00,7.00,0.0,4.00,52.00
off_median,-41.00,-18.00,0.0,0.0,-6.00,0.0,0.0,-12.00,0.0,-5.00,0.0,31.00,-15.00,0.0,-12.50,12.00
off_mean,-20.80,-21.40,0.0,0.0,-4.94,0.0,0.0,-11.69,0.0,-4.94,0.0,31.00,-11.25,0.0,-15.75,9.74
off_iqr,39.00,22.00,0.0,0.0,10.50,0.0,0.0,18.50,0.0,20.00,0.0,0.00,15.50,0.0,31.50,42.50
off_std,42.20,14.84,0.0,0.0,10.04,0.0,0.0,14.99,0.0,16.77,0.0,33.94,10.73,0.0,17.49,27.05
vel_min,23.00,30.00,0.0,0.0,88.00,0.0,0.0,32.00,0.0,16.00,0.0,8.00,89.00,0.0,94.00,30.00
vel_max,34.00,39.00,0.0,0.0,111.00,0.0,0.0,117.00,0.0,42.00,0.0,34.00,114.00,0.0,116.00,46.00
vel_median,29.50,38.00,0.0,0.0,101.00,0.0,0.0,97.50,0.0,33.00,0.0,21.00,101.50,0.0,111.50,37.00


Showing stats for: 42


...

bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
off_min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,0.0,0.0,0.0,0.0,0.0
off_max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,0.0,0.0,0.0,0.0,0.0
off_median,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,0.0,0.0,0.0,0.0,0.0
off_mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,0.0,0.0,0.0,0.0,0.0
off_iqr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vel_min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0,0.0
vel_max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0,0.0
vel_median,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0,0.0
vel_mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,0.0,0.0,0.0,0.0,0.0


Showing stats for: 49


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,0.0,0.0,0.0,0.0,0.0
off_min,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-37.00,0.0,0.0,0.0,0.0,0.0
off_max,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-16.00,0.0,0.0,0.0,0.0,0.0
off_median,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-33.00,0.0,0.0,0.0,0.0,0.0
off_mean,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-28.67,0.0,0.0,0.0,0.0,0.0
off_iqr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.50,0.0,0.0,0.0,0.0,0.0
off_std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.15,0.0,0.0,0.0,0.0,0.0
vel_min,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.00,0.0,0.0,0.0,0.0,0.0
vel_max,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.00,0.0,0.0,0.0,0.0,0.0
vel_median,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.00,0.0,0.0,0.0,0.0,0.0


Showing stats for: 51


bar_beat_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
hits,16.00,0.0,16.00,3.00,16.00,0.0,16.00,0.0,17.00,0.0,12.00,0.0,16.00,0.0,16.00,0.0
off_min,-29.00,0.0,-41.00,-12.00,-30.00,0.0,-29.00,0.0,-33.00,0.0,-32.00,0.0,-37.00,0.0,-39.00,0.0
off_max,13.00,0.0,11.00,-6.00,20.00,0.0,27.00,0.0,23.00,0.0,19.00,0.0,12.00,0.0,11.00,0.0
off_median,-2.50,0.0,-11.50,-8.00,-2.00,0.0,3.50,0.0,0.00,0.0,2.00,0.0,-11.50,0.0,-5.50,0.0
off_mean,-2.94,0.0,-11.06,-8.67,-3.69,0.0,1.38,0.0,-2.00,0.0,-0.08,0.0,-9.88,0.0,-7.38,0.0
off_iqr,16.50,0.0,14.00,3.00,9.50,0.0,14.50,0.0,18.00,0.0,14.00,0.0,18.00,0.0,24.00,0.0
off_std,10.83,0.0,12.40,3.06,12.21,0.0,12.88,0.0,14.54,0.0,13.26,0.0,12.27,0.0,14.75,0.0
vel_min,38.00,0.0,31.00,39.00,48.00,0.0,36.00,0.0,0.00,0.0,36.00,0.0,33.00,0.0,39.00,0.0
vel_max,60.00,0.0,51.00,49.00,71.00,0.0,48.00,0.0,64.00,0.0,56.00,0.0,73.00,0.0,83.00,0.0
vel_median,51.00,0.0,45.00,45.00,57.50,0.0,42.50,0.0,52.00,0.0,46.00,0.0,54.00,0.0,51.50,0.0
